In [1]:
import os
os.chdir("/app")

In [2]:
import pandas as pd
from pathlib import Path
from nbgrader.api import Gradebook
from nbgrader.apps import NbGraderAPI
from traitlets.config import Config
from tqdm.auto import tqdm

In [3]:
COURSE_ID = "itmo_recsys_2025_spring"

In [4]:
config = Config()
config.CourseDirectory.course_id = COURSE_ID

In [5]:
nbg = NbGraderAPI(config=config)

In [6]:
ASSIGNMENT = "hw_2"

In [7]:
students = nbg.get_students()
student_ids = [student["id"] for student in students]
len(student_ids)

47

In [8]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
type(autograded_student_ids), len(autograded_student_ids)

(set, 39)

In [10]:
not_autograded_student_ids = [s_id for s_id in student_ids if s_id not in autograded_student_ids]
len(not_autograded_student_ids)

8

In [11]:
not_autograded_student_ids

['koshkidadanet',
 'Kraagger-del',
 'limmark21',
 '1MaxOn1',
 'savoskinsemyonq',
 'stasstaf',
 'dimajyg',
 'nonodoubt']

In [12]:
nbg.autograde(assignment_id=ASSIGNMENT, student_id="koshkidadanet", force=False, create=False)

{'success': False,
 'error': 'Traceback (most recent call last):\n  File "/usr/local/lib/python3.10/site-packages/nbgrader/utils.py", line 542, in capture_log\n    app.start()\n  File "/usr/local/lib/python3.10/site-packages/nbgrader/converters/base.py", line 124, in start\n    self.init_notebooks()\n  File "/usr/local/lib/python3.10/site-packages/nbgrader/converters/base.py", line 182, in init_notebooks\n    raise NbGraderException(msg)\nnbgrader.converters.base.NbGraderException: No notebooks were matched by \'/app/submitted/koshkidadanet/hw_2\'\n',
 'log': "[ERROR] No notebooks were matched by '/app/submitted/koshkidadanet/hw_2'\n"}

In [43]:
not_autograded_student_ids.index("ONEPANTSU")

4

In [44]:
results[4]

{'success': True,
 'log': '[INFO] Skipping existing assignment: /app/autograded/ONEPANTSU/hw_2\n'}

In [37]:
results = []
for student_id in tqdm(not_autograded_student_ids):
    res = nbg.autograde(assignment_id=ASSIGNMENT, student_id=student_id, force=False, create=False)
    results.append(res)

  0%|          | 0/42 [00:00<?, ?it/s]

In [38]:
successes = [res for res in results if res["success"]]
failures = [res for res in results if not res["success"]]

graded = [res for res in successes if "[INFO] Autograding" in res["log"]]
skipped = [res for res in successes if res["log"].startswith("[INFO] Skipping")]


print("Total: ", len(results))
print("failures: ", len(failures))
print("successes: ", len(successes))
print("graded: ", len(graded))
print("skipped: ", len(skipped))

Total:  42
failures:  8
successes:  34
graded:  0
skipped:  34


In [46]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
type(autograded_student_ids), len(autograded_student_ids)

(set, 39)

In [47]:
for s_id in tqdm(autograded_student_ids):
    nbg.generate_feedback(assignment_id=ASSIGNMENT, student_id=s_id, force=False)

  0%|          | 0/39 [00:00<?, ?it/s]

In [13]:
from pathlib import Path
import re

In [14]:
def get_scores():
    scores = {}
    for student_id in os.listdir("/app/feedback"):
        # print(student_id)
        path = Path("/app/feedback") / student_id / ASSIGNMENT / "metrics.html"
        try:
            text = path.read_text()
        except FileNotFoundError as e:
            print(e)
            continue
        
        total_match = re.search(r"metrics \(Score: (\d+\.\d+ / \d+\.\d+)\)", text)
        total_score = total_match.groups()[0]
    
        cell_scores = re.findall(r'<li><a href="#cell-\w+?">Test cell</a> \(Score: (\d+.\d+ / \d+.\d+)\)</li>', text)    
    
        scores[student_id] = {"total": total_score, "cells": cell_scores}
    return scores

In [15]:
scores = get_scores()

In [16]:
scores["stasstaf"]

{'total': '8.0 / 15.0',
 'cells': ['1.0 / 1.0',
  '1.0 / 1.0',
  '1.0 / 1.0',
  '1.0 / 1.0',
  '0.0 / 1.0',
  '0.0 / 1.0',
  '0.0 / 2.0',
  '0.0 / 3.0',
  '2.0 / 2.0',
  '2.0 / 2.0']}

In [17]:
names = [
    "precision_and_recall",  
    "ap_at_3_for_user_2", 
    "map_at_3", 
    "dcg_at_3_for_user_2", 
    "idcg_at_3_for_user_2", 
    "ndcg_at_3",
    "weighted_recall_correct",
    "weighted_recall_efficient",
    "model_A",
    "model_B"
]    

In [18]:
def parse_scores(scores):
    total_scores = {}
    students_feedback = []
    for student_id, feedback in scores.items():
        student_score = int(float(feedback["total"].split(" /")[0]))
        total_scores[student_id] = student_score
        feedbacks = [f"total: {feedback['total']}"]
        for name, result_score in zip(names, feedback["cells"]):
            feedbacks.append(f"{name}: {result_score}")
        students_feedback.append({
            "student_id": student_id,
            "feedback": ", ".join(feedbacks),
            "score": student_score
        })
    return total_scores, students_feedback

In [19]:
total_scores, students_feedback = parse_scores(scores)

In [24]:
total_scores["stasstaf"], [sf for sf in students_feedback if sf["student_id"] == "stasstaf"]

(8,
 [{'student_id': 'stasstaf',
   'feedback': 'total: 8.0 / 15.0, precision_and_recall: 1.0 / 1.0, ap_at_3_for_user_2: 1.0 / 1.0, map_at_3: 1.0 / 1.0, dcg_at_3_for_user_2: 1.0 / 1.0, idcg_at_3_for_user_2: 0.0 / 1.0, ndcg_at_3: 0.0 / 1.0, weighted_recall_correct: 0.0 / 2.0, weighted_recall_efficient: 0.0 / 3.0, model_A: 2.0 / 2.0, model_B: 2.0 / 2.0',
   'score': 8}])

In [102]:
manual_feedback = [
    {"student_id": "ONEPANTSU",
     "feedback": "Invalid notebook error: https://github.com/ONEPANTSU/recommendation-service/blob/hw_2/notebooks/hw_2/metrics.ipynb",
     "score": 0,},
    {"student_id": "savoskinsemyonq",
     "feedback": "404 error: https://github.com/savoskinsemyonq/RecSys_ITMO/blob/hw_2/notebooks/hw_2/metrics.ipynb",
     "score": 0,
    },
]

In [103]:
import pandas as pd
df = pd.DataFrame(students_feedback+manual_feedback)
df.to_csv("hw2_feedback.csv", index=False)

In [104]:
df.head()

,student_id,feedback,score
0,ElenaSuhova,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
1,nbusko,"total: 8.0 / 15.0, precision_and_recall: 1.0 /...",8
2,Darinochka,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
3,7Askar7,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
4,NuatStanskiy,"total: 8.0 / 15.0, precision_and_recall: 1.0 /...",8


# Regrade (after tests update)

In [115]:
students_to_regrade = df[df["score"]<15]["student_id"].values
students_to_regrade = [x for x in students_to_regrade if x not in ("ONEPANTSU", "savoskinsemyonq")]

In [116]:
results = []
for student_id in tqdm(students_to_regrade):
    res = nbg.autograde(assignment_id=ASSIGNMENT, student_id=student_id, force=True, create=False)
    results.append(res)

  0%|          | 0/25 [00:00<?, ?it/s]

In [117]:
successes = [res for res in results if res["success"]]
failures = [res for res in results if not res["success"]]

graded = [res for res in successes if "[INFO] Autograding" in res["log"]]
skipped = [res for res in successes if res["log"].startswith("[INFO] Skipping")]


print("Total: ", len(results))
print("failures: ", len(failures))
print("successes: ", len(successes))
print("graded: ", len(graded))
print("skipped: ", len(skipped))

Total:  25
failures:  0
successes:  25
graded:  25
skipped:  0


In [118]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
type(autograded_student_ids), len(autograded_student_ids)

(set, 39)

In [119]:
for s_id in tqdm(autograded_student_ids):
    nbg.generate_feedback(assignment_id=ASSIGNMENT, student_id=s_id, force=True)

  0%|          | 0/39 [00:00<?, ?it/s]

In [128]:
scores = get_scores()

In [126]:
scores

In [129]:
total_scores, students_feedback = parse_scores(scores)

In [133]:
total_scores

{'ElenaSuhova': 15,
 'nbusko': 13,
 'Darinochka': 15,
 '7Askar7': 15,
 'NuatStanskiy': 8,
 'DmitryRedko': 15,
 'qw1zzard': 15,
 'EugenePWN': 13,
 'FanisNgv': 8,
 'n0tmyself': 10,
 'martynov-dm': 11,
 'popov101': 8,
 'smalda': 15,
 'katimanova': 13,
 'anaaaiva': 13,
 'khrstln': 6,
 'timsmr': 12,
 'wilfordaf': 11,
 'semyondipner': 15,
 'nsgorbunov': 15,
 'BulatMaratovich': 9,
 'Qeshtir': 10,
 'Postironia1': 8,
 'V-slav-github': 13,
 'alexbuyan': 13,
 'ArtemIgnatenko': 9,
 'nbelyaev1': 15,
 'Laitielly': 10,
 'LeftGoga': 15,
 'ShamilNur': 11,
 'Mihail-Olegovich': 13,
 'UsefulTornado': 8,
 'Evgenii-Iurin': 8,
 'PaulRychkov': 10,
 'GlebIsrailevich': 15,
 'pavelkochkin1': 11}

In [132]:
students_feedback

[{'student_id': 'ElenaSuhova',
  'feedback': 'total: 15.0 / 15.0, precision_and_recall: 1.0 / 1.0, ap_at_3_for_user_2: 1.0 / 1.0, map_at_3: 1.0 / 1.0, dcg_at_3_for_user_2: 1.0 / 1.0, idcg_at_3_for_user_2: 1.0 / 1.0, ndcg_at_3: 1.0 / 1.0, weighted_recall_correct: 2.0 / 2.0, weighted_recall_efficient: 3.0 / 3.0, model_A: 2.0 / 2.0, model_B: 2.0 / 2.0',
  'score': 15},
 {'student_id': 'nbusko',
  'feedback': 'total: 13.0 / 15.0, precision_and_recall: 1.0 / 1.0, ap_at_3_for_user_2: 0.0 / 1.0, map_at_3: 0.0 / 1.0, dcg_at_3_for_user_2: 1.0 / 1.0, idcg_at_3_for_user_2: 1.0 / 1.0, ndcg_at_3: 1.0 / 1.0, weighted_recall_correct: 2.0 / 2.0, weighted_recall_efficient: 3.0 / 3.0, model_A: 2.0 / 2.0, model_B: 2.0 / 2.0',
  'score': 13},
 {'student_id': 'Darinochka',
  'feedback': 'total: 15.0 / 15.0, precision_and_recall: 1.0 / 1.0, ap_at_3_for_user_2: 1.0 / 1.0, map_at_3: 1.0 / 1.0, dcg_at_3_for_user_2: 1.0 / 1.0, idcg_at_3_for_user_2: 1.0 / 1.0, ndcg_at_3: 1.0 / 1.0, weighted_recall_correct: 2.0 /

In [130]:
df = pd.DataFrame(students_feedback+manual_feedback)
df.to_csv("hw2_feedback.csv", index=False)

In [131]:
df

,student_id,feedback,score
0,ElenaSuhova,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
1,nbusko,"total: 13.0 / 15.0, precision_and_recall: 1.0 ...",13
2,Darinochka,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
3,7Askar7,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
4,NuatStanskiy,"total: 8.0 / 15.0, precision_and_recall: 1.0 /...",8
5,DmitryRedko,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
6,qw1zzard,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
7,EugenePWN,"total: 13.0 / 15.0, precision_and_recall: 1.0 ...",13
8,FanisNgv,"total: 8.0 / 15.0, precision_and_recall: 1.0 /...",8
9,n0tmyself,"total: 10.0 / 15.0, precision_and_recall: 1.0 ...",10
